# Instalando Bibliotecas Importantes

In [ ]:
!pip install torchmetrics

## Importando Principais Bibliotecas

In [ ]:
import torch
import torchvision
import numpy as np
import random
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import MobileNet_V3_Large_Weights
from torch.optim.lr_scheduler import OneCycleLR
import matplotlib.pyplot as plt
import torch.serialization
import torchmetrics
import torch.nn as nn

### Fixando seed

In [ ]:
np.random.seed(42)
torch.random.manual_seed(42)
random.seed(42)

### Montando Drive com o dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


data = '/content/drive/MyDrive/DatasetPlacas_WandersonJunior'

## Criando arquivo de anotações em csv e associação de classe numérica com string
- Cada linha do csv contém como colunas o nome da imagem e sua label correspondente

- As classes foram convertidas da seguinte forma:

```
R-1:   0
A-18:  1
A-32:  2
R-6a:  3
R-3:   4
```

In [ ]:
data_csv = []
class_csv = []


for id, classname in enumerate(os.listdir(data)):
  class_csv.append([classname,id])
  classe_imgs = os.path.join(data,classname)
  if os.path.isdir(classe_imgs):
    for img_name in os.listdir(classe_imgs):
      data_csv.append([img_name, id])

df_class = pd.DataFrame(class_csv,columns=['classname','id'])
df_class.to_csv('/content/drive/MyDrive/DatasetPlacas_WandersonJunior/class_idx.csv',index=False)
df = pd.DataFrame(data_csv,columns=['filename','label'])
df.to_csv('/content/drive/MyDrive/DatasetPlacas_WandersonJunior/labels.csv',index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DatasetPlacas_WandersonJunior/labels.csv')

In [ ]:
data ='/content/drive/MyDrive/data'

## Criando Classe Dataset
- que lê as imagens do dataset dentro de uma pasta /data e as anotações dentro do arquivo /labels.csv

In [ ]:
class PlacasDataset(Dataset):
  def __init__(self, img_dir, label_file, transform=None):
    self.labels = label_file
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.labels)


  def __getitem__(self,idx):
    img_path = os.path.join(self.img_dir, self.labels.iloc[idx,0])
    image = Image.open(img_path).convert('RGB')
    label = self.labels.iloc[idx, 1]

    if self.transform:
      image = self.transform(image)

    return image, label

### Aplicando transformações nas imagens
  - redução das imagens para um tamanho padronizado (para caber na gpu)
  - Aplicação de rotações, translações e ruido gaussiano
  - Aplicação da normalização no final baseado na normalização da ImageNet

In [ ]:
transform = transforms.Compose([
    transforms.Resize((512,512)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
    transforms.RandomApply([transforms.GaussianBlur(5)],p=0.3),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### Separação em treino, teste e validação
- pego o dataframe criado a partir do csv de anotações e particiono ele em 3 dataframes com 60% dos dados no dataframe de treino, 20% no de teste e 20% no de validação

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

total = len(df)
train_end = int(0.6 * total)
val_end = train_end + int(0.2 * total)

df_train = df[:train_end]
df_val   = df[train_end:val_end]
df_test  = df[val_end:]

train = PlacasDataset(data,df_train,transform)
val = PlacasDataset(data,df_val,transform)
test = PlacasDataset(data,df_test,transform)


## Loop de treino
- roda o loop de treino padrão do pytorch com validação em todas as épocas
- métrica de treino f1-score
- salva modelo com melhor f1-score na validação

In [ ]:
def train_Classifier(model, epochs, optimizer, criterion, scheduler, f1,train,val,train_losses, val_losses):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model.to(device)
  score = 0.0

  trainloader = DataLoader(train,batch_size=8,num_workers=2)
  valloader = DataLoader(val,batch_size=1,num_workers=2)


  for epoch in range(epochs):
    model.train()
    trainloss = 0
    for i, data in enumerate(trainloader,0):
        img, label = data
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()

        outputs = model(img)
        loss = criterion(outputs,label)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)

        trainloss += loss.item()

        lr = scheduler.get_last_lr()[0]

        print(f"[Epoch {epoch+1}/{epochs}, Batch {i}] Loss: {trainloss:.4f} | LR: {lr:.6f}")

    loss_avg = trainloss / len(trainloader)
    print(f"loss_avg: {loss_avg:.4f}")

    train_losses.append(loss_avg)



    with torch.no_grad():
      model.eval()
      val_loss = 0.0
      f1.reset()
      for img_val, label_val in valloader:
        img_val = img_val.to(device)
        label_val = label_val.to(device)
        outputs = model(img_val)

        indexes, predicao = torch.max(outputs, 1)

        f1.update(predicao, label_val)

        loss_val = criterion(outputs, label_val)

        val_loss += loss_val.item()

      final_f1 = f1.compute()

      print(f"f1 score de validação: {final_f1.item():.4f}")
      print(f"val_loss: {val_loss:.4f}")

      val_loss_avg = val_loss / len(valloader)
      val_losses.append(val_loss_avg)

      if score < final_f1:
        score = final_f1
        torch.save(model, f"/content/drive/MyDrive/Resultados/melhor_modelo_{epoch}.pth")




      model.train()

# Definição do modelo para treino
- Ajusta a MobileNet para classificação de 5 classes
- define épocas de treino
- otimizador: AdamW
- Scheduler: OneCycleLR

In [ ]:
weights = MobileNet_V3_Large_Weights.DEFAULT
model = torchvision.models.mobilenet_v3_large(weights=weights)
model.classifier[-1] = nn.Linear(1280, 5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 30
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
loss = torch.nn.CrossEntropyLoss()

scheduler = OneCycleLR(optimizer,
                       max_lr=1e-3,
                       total_steps=(len(train)//8) * 30)

f1 = torchmetrics.F1Score('multiclass', num_classes=5,average='macro').to(device)

train_losses = []
val_losses = []

train_Classifier(model, epochs, optimizer, loss, scheduler,f1, train,val,train_losses,val_losses)

### Plotagem das Curvas de Loss de Treino e Validação

In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss durante o Treinamento')
plt.show()

## inferência

Cálculo das métricas f1-score macro, accuracy e precision para o conjunto de teste com o melhor modelo obtido no treino

In [ ]:
model = torch.load("/content/drive/MyDrive/Resultados/melhor_modelo_10.pth", weights_only=False)

accuracy = torchmetrics.Accuracy('multiclass',num_classes=5).to(device)
precision = torchmetrics.Precision('multiclass',num_classes=5).to(device)

f1.reset()
accuracy.reset()
precision.reset()


testloader = DataLoader(test,batch_size=1,num_workers=2)

with torch.no_grad():
  for image_test, label_test in testloader:
    image = image_test.to(device)
    label = label_test.to(device)

    outputs = model(image)
    preds = torch.argmax(outputs, dim=1)

    f1.update(preds,label)
    accuracy.update(preds,label)
    precision.update(preds,label)


f1_test = f1.compute()
accuracy_test = accuracy.compute()
precision_test = precision.compute()

print(f"F1-Score: {f1_test}")
print(f"Acurácia: {accuracy_test}")
print(f"Precision: {precision_test}")